In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.chdir('/Users/sebas/projects/metabolomics')
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

In [ ]:
from src.transformers.sklearn_model import SklearnModel
import gensim
from src.load_data import LoadData
from src.config import Config
from tqdm import tqdm
from src.loader_saver import LoaderSaver
import itertools
import numpy as np
from scipy.stats import spearmanr
import dill
from src.plotting import Plotting
from src.load_data import LoadData
from src.molecule_pairs_opt import MoleculePairsOpt
from sklearn.model_selection import train_test_split
from src.train_utils import TrainUtils
from src.preprocessor import Preprocessor
import pickle
import sys
from src.config import Config
from src.parser import Parser
from datetime import datetime
from src.loader_saver import LoaderSaver
from src.molecular_pairs_set import MolecularPairsSet
from scipy.stats import spearmanr
from src.transformers.embedder import Embedder
from src.transformers.encoder import Encoder
import matplotlib.pyplot as plt
from src.transformers.CustomDatasetEncoder import CustomDatasetEncoder
from src.transformers.load_data_encoder import LoadDataEncoder
from torch.utils.data import DataLoader
from src.analog_discovery.cosine_similarity import CosineSimilarity
from rdkit import Chem
from matchms.importing import load_from_mgf,load_from_msp
from matchms.similarity import ModifiedCosine
from src.spec2vec_comparison import Spec2VecComparison
from spec2vec import Spec2Vec
import tensorflow as tf

## params

In [ ]:
ls /Users/sebas/projects/data/

In [ ]:
data_folder= '/Users/sebas/projects/data/'
gnps_path =  data_folder + 'ALL_GNPS_NO_PROPOGATED_wb.mgf'
janssen_path = data_folder + 'drug_plus.mgf'
nist_path = data_folder + 'hr_msms_nist_all.MSP'
output_janssen_file= data_folder + 'all_spectrums_janssen.pkl'
dataset_path= data_folder +'merged_gnps_nist_20240311_unique_smiles_1_million.pkl'
model_path = data_folder + 'best_model_20240319_v2_512u_5_layers.ckpt'
model_spec2vec_file = data_folder + 'spec2vec_AllPositive_ratio05_filtered_201101_iter_15.model'
model_ms2d_file = data_folder + 'ms2deepscore_positive_10k_1000_1000_1000_500.hdf5'

In [ ]:
config=Config()

In [ ]:
config.D_MODEL=512
config.N_LAYERS=5

In [ ]:
similarity_name= 'spec2vec'

## open janssen data

In [ ]:
all_spectrums_janssen_matchms = [s for s in load_from_mgf(janssen_path)]

In [ ]:
loader_saver = LoaderSaver(
        block_size=100,
        pickle_nist_path='',
        pickle_gnps_path='',
        pickle_janssen_path=output_janssen_file,
    )

In [ ]:
all_spectrums_janssen_su = loader_saver.get_all_spectrums(
            janssen_path,
            100000000,
            use_tqdm=True,
            use_nist=False,
            config=config,
            use_janssen=True,
        )

## open ref data

In [ ]:
with open(dataset_path, 'rb') as file:
            dataset = dill.load(file)

In [ ]:
all_spectrums_reference_su= dataset['molecule_pairs_train'].spectrums_original +\
                    dataset['molecule_pairs_val'].spectrums_original + \
                        dataset['molecule_pairs_test'].spectrums_original

In [ ]:
all_spectrums_gnps_matchms = [s for s in load_from_mgf(gnps_path)]

In [ ]:
all_spectrums_nist_matchms=  [s for s in load_from_msp(nist_path)]

In [ ]:
all_spectrums_reference_matchms = all_spectrums_gnps_matchms + all_spectrums_nist_matchms

In [ ]:
#compute all the hashes from the datasets
target_hashes = [s.spectrum_hash for s in all_spectrums_reference_su]

In [ ]:
matchms_hashes=[]
for s in  tqdm(all_spectrums_reference_matchms):
    matchms_hashes.append(s.spectrum_hash())

In [ ]:
len(target_hashes)

In [ ]:
len(all_spectrums_reference_matchms)

In [ ]:
matchms_hashes.index(target_hashes[0])

In [ ]:
indexes_matched = [matchms_hashes.index(t) for t in tqdm(target_hashes)]

In [ ]:
all_spectrums_reference = [all_spectrums_reference_matchms[index] for index in indexes_matched]

In [ ]:
for i,(s_ref, s_su) in enumerate(zip(all_spectrums_reference, all_spectrums_reference_su)):
    new_metadata=s_ref.metadata.copy()
    new_metadata['smiles']=s_su.smiles
    all_spectrums_reference[i].metadata=new_metadata

## Filter spectra from Janssen

In [ ]:
su_hashes = [s.spectrum_hash for s in all_spectrums_janssen_su]

In [ ]:
all_spectrums_janssen = [s for s in all_spectrums_janssen_matchms if s.spectrum_hash() in su_hashes]

## Find those instances that are not in reference

In [ ]:
canon_smiles_reference = [Chem.CanonSmiles(s.metadata['smiles']) for s in all_spectrums_reference]
canon_smiles_janssen =   [Chem.CanonSmiles(s.metadata['smiles']) for s in all_spectrums_janssen]
janssen_indexes_in_ref= [i for i,s in enumerate(canon_smiles_janssen) if s in canon_smiles_reference]
janssen_indexes_not_in_ref = [i for i,s in enumerate(canon_smiles_janssen) if s not in canon_smiles_reference]

In [ ]:
len(janssen_indexes_in_ref),len(janssen_indexes_not_in_ref)

In [ ]:
all_spectrums_janssen = [all_spectrums_janssen[index] for index in janssen_indexes_not_in_ref]

## load model

In [ ]:
#encoder= Encoder(model_path, D_MODEL=int(config.D_MODEL),N_LAYERS=int(config.N_LAYERS))

In [ ]:
#similarity_model =ModifiedCosine(tolerance=0.1)

In [ ]:
if similarity_name=='spec2vec':
    model = gensim.models.Word2Vec.load(model_spec2vec_file)
    similarity_model= Spec2Vec(
                model=model, intensity_weighting_power=0.5, allowed_missing_percentage=100.0
            )
    PREPROCESS_SPECTRUMS=True
elif similarity_name=='modified_cosine':
    similarity_model =ModifiedCosine(tolerance=0.1)
    PREPROCESS_SPECTRUMS=True
elif similarity_name == 'ms2deepscore':
    model_ms2d = load_model(model_ms2d_file)
    similarity_model= MS2DeepScore(model_ms2d)

## Preprocessed spectrums

In [ ]:
preprocessed_all_spectrums_janssen =all_spectrums_janssen.copy()
if PREPROCESS_SPECTRUMS:
    for i,s in tqdm(enumerate(preprocessed_all_spectrums_janssen)):
        preprocessed_all_spectrums_janssen[i] = Spec2VecComparison.spectrum_processing(preprocessed_all_spectrums_janssen[i])
preprocessed_all_spectrums_janssen = [s for s in preprocessed_all_spectrums_janssen if ((s is not None)and (s.metadata['precursor_mz']>0))]

In [ ]:
preprocessed_all_spectrums_reference =all_spectrums_reference.copy()
if PREPROCESS_SPECTRUMS:
    for i,s in tqdm(enumerate(preprocessed_all_spectrums_reference)):
        preprocessed_all_spectrums_reference[i] = Spec2VecComparison.spectrum_processing(preprocessed_all_spectrums_reference[i])
preprocessed_all_spectrums_reference = [s for s in preprocessed_all_spectrums_reference if ((s is not None)and (s.metadata['precursor_mz']>0))]

In [ ]:
#with open('preprocessed_all_spectrums_reference.pkl', 'wb') as file:
#        dictionary={'preprocessed_all_spectrums_reference':preprocessed_all_spectrums_reference}
#        dill.dump(dictionary,file)

#with open('preprocessed_all_spectrums_reference.pkl', 'rb') as file:
#        preprocessed_all_spectrums_reference=dill.load(file)['preprocessed_all_spectrums_reference']

## compute similarities

In [ ]:
from matchms import calculate_scores
if similarity_name != 'ms2deepscore':
    results_scores = calculate_scores(
                        preprocessed_all_spectrums_reference, preprocessed_all_spectrums_janssen, similarity_model
                    )
else:
    with tf.device('/device:CPU:0'): #execute on cpu
        results_scores = calculate_scores(
                        preprocessed_all_spectrums_reference, preprocessed_all_spectrums_janssen, similarity_model
                    )

In [ ]:
results_scores

In [ ]:
results_tuple = [results_scores.scores_by_query(s, name='Spec2Vec', sort=True)[0] for s in preprocessed_all_spectrums_janssen]
spectrums_retrieved = [r[0] for r in results_tuple]
max_sim = [r[1] for r in results_tuple]

In [ ]:
#spectrums_retrieved = [results_scores.scores_by_query(s, name='Spec2Vec', sort=True)[0][0] \
#                       for s in preprocessed_all_spectrums_janssen]

In [ ]:
#max_sim = [results_scores.scores_by_query(s, name='Spec2Vec', sort=True)[0][1] \
#                       for s in preprocessed_all_spectrums_janssen]

## Based on the similarities compute the similarity score of the match spectrum

In [ ]:
smiles_retrieved = [s.metadata['smiles'] for s in spectrums_retrieved]

In [ ]:
smiles_janssen = [s.metadata['smiles'] for s in preprocessed_all_spectrums_janssen]

In [ ]:
from src.tanimoto import Tanimoto

In [ ]:
tanimoto_retrieved = [Tanimoto.compute_tanimoto_from_smiles(s0,s1) for s0, s1 in zip(smiles_janssen, smiles_retrieved)]

In [ ]:
_=plt.hist(max_sim, color='r',bins=10)
plt.xlabel('maximum predicted similarity found')
plt.ylabel('freq')
plt.grid()

In [ ]:
plt.hist(tanimoto_retrieved)
plt.grid()
plt.xlabel('tanimoto similarity with reference spectra')
plt.ylabel('frequency')

In [ ]:
tanimoto_retrieved= np.array(tanimoto_retrieved)

In [ ]:
plt.scatter(tanimoto_retrieved, max_sim, alpha=1)
plt.xlim([0,1])
plt.ylim([0,1])
plt.ylabel('prediction')
plt.xlabel('tanimoto')

## Check which is the spectra that has wrong predictions

In [ ]:
from rdkit import Chem
from rdkit.Chem import rdFMCS
def calculate_mcs_similarity(smiles1, smiles2):
    mol1 = Chem.MolFromSmiles(smiles1)
    mol2 = Chem.MolFromSmiles(smiles2)
    
    # Perform MCS (Maximum Common Substructure) search
    mcs = rdFMCS.FindMCS([mol1, mol2])

    # Get SMARTS pattern from MCS result
    mcs_smarts = Chem.MolToSmarts(mcs.queryMol)
    
    # Calculate Tanimoto-like similarity
    mcs_mol = Chem.MolFromSmarts(mcs_smarts)
    #mcs_count = len(Chem.GetMolFrags(mcs_mol))
    mcs_count = mcs_mol.GetNumAtoms()
    similarity = mcs_count / (mol1.GetNumAtoms() + mol2.GetNumAtoms() - mcs_count)

    return similarity, mcs_mol
from rdkit.Chem import rdRascalMCES
def calculate_mces_sim(smiles1, smiles2, similarity_threshold=0.7):

    ad1 = Chem.MolFromSmiles(smiles1)
    ad2 = Chem.MolFromSmiles(smiles2)
    opts = rdRascalMCES.RascalOptions()
    opts.similarityThreshold = similarity_threshold
    opts.returnEmptyMCES = True
    results = rdRascalMCES.FindMCES(ad1, ad2, opts)
    if len(results) != 0:
        similarity_tier1= results[0].tier1Sim
        similarity_tier2= results[0].tier2Sim
    
        if similarity_tier2 != -1:
            return similarity_tier2 #if the lower threshold is not surpassed
        else:
            return similarity_tier1
    else:
        return None

In [ ]:
len(all_spectrums_janssen)

In [ ]:
len(spectrums_retrieved)

In [ ]:
target_spectra=0

In [ ]:
len(max_sim)

In [ ]:
max_sim[target_spectra]

In [ ]:
tanimoto_retrieved[target_spectra]

In [ ]:
## mcs
sim_mcs, mol_mcs=calculate_mcs_similarity(smiles1=all_spectrums_janssen[target_spectra].metadata['smiles'], 
                         smiles2=spectrums_retrieved[target_spectra].metadata['smiles'])
sim_mcs

In [ ]:
sim_mces = calculate_mces_sim(smiles1=all_spectrums_janssen[target_spectra].metadata['smiles'], 
                         smiles2=spectrums_retrieved[target_spectra].metadata['smiles'])
sim_mces

In [ ]:
mol_mcs

In [ ]:
## plot the molecules

In [ ]:
Chem.CanonSmiles(all_spectrums_janssen[target_spectra].metadata['smiles'])

In [ ]:
from rdkit import Chem
mol_janssen = Chem.MolFromSmiles(all_spectrums_janssen[target_spectra].metadata['smiles'])
mol_janssen

In [ ]:
Chem.CanonSmiles(spectrums_retrieved[target_spectra].metadata['smiles'])

In [ ]:
mol_ref = Chem.MolFromSmiles(spectrums_retrieved[target_spectra].metadata['smiles'])
mol_ref

## FIND SIMILARITY BASED ON MCS

In [ ]:
len(smiles_retrieved)

In [ ]:
len(smiles_janssen)

In [ ]:
mces_sims=[]
for s0,s1 in tqdm(zip(smiles_janssen, smiles_retrieved)):
    similarity= calculate_mces_sim(s0, s1)
    mces_sims.append(similarity)

In [ ]:
# Specify the bin width
bin_width = 0.1

# Calculate the number of bins based on the data range and bin width
bins = np.arange(0, 1 + bin_width, bin_width)

In [ ]:
plt.hist(tanimoto_retrieved,alpha=0.5, label='tanimoto sim.', density=True, bins=bins)
plt.hist([m for m in mces_sims if m is not None],alpha=0.5, label='mces sim.', density=True, bins=bins)
plt.xlabel('similarity')
plt.ylabel('density')
plt.legend()
plt.grid()

## saving of results

In [ ]:
results ={ 'preprocessed_all_spectrums_janssen':preprocessed_all_spectrums_janssen,
            'smiles_janssen':smiles_janssen,
          'smiles_retrieved':smiles_retrieved,
          'spectrums_retrieved':spectrums_retrieved,
            'tanimoto_retrieved':tanimoto_retrieved,
          'max_sim':max_sim,
          'mces_retrieved':mces_sims}

In [ ]:
with open('./notebooks/discovery_search/results/'+similarity_name + '_results_analog_discovery_unknwon_compounds.pkl', 'wb') as f:
    dill.dump(results, f)